<a href="https://colab.research.google.com/github/HwangHanJae/recommender_system/blob/main/book_Recommender_Systems/Model_Base_NMF_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NMF(Non-negative Matrix Factorization)
NMF는 음수 미포함 행렬 분해(비음행렬 인수분해)라고 하여 말 그대로 음수를 포함하지 않는 행렬 $X$를 음수를 포함하지 않는 행렬 $W$와 $H$의 곱으로 분해하는 알고리즘입니다.

$$X = WH$$

NMF와 관련하여 참고한 자료는 다음과 같습니다.  
참고링크1 : https://angeloyeo.github.io/2020/10/15/NMF.html  
참고링크2 : https://github.com/ahmadvh/Non-Negative-Matrix-factorization---Implemented-in-python


여기서는 음수를 포함하지 않는 평점을 가진 행렬을 비음평점 행렬 혹은 비음행렬이라고 표현하겠습니다.

비음평점 행렬은 암시적 피드백 데이터세트에 일반적으로 사용됩니다.

암시적 피드백 데이터 세트의 예는 다음과 같습니다.
1. 사용자가 특정 아이템에 대하여 구매 하였을 때 
2. 웹에서 사용자가 특정 아이템에 대해서 클릭 하였을 때
3. 페이스북의 "좋아요" 버튼을 선택하였을 때

위의 예처럼 사용자의 표현은 음수가 아닌 임의의 숫자로 표현할 수 있고, 이러한 숫자는 아이템에 대해서 호감도를 명시하지만 비호감도를 나타내는 것은 아닙니다.

즉 암시적 피드백 데이터 세트는 신뢰를 나타내고 명시적 피드백의 경우는 선호를 나타냅니다.

우리의 목적은 $WH$가 최대한 $X$와 가깝게 만드는 것입니다.
최대한 $X$와 가깝다 라는 의미는 $WH$와 $X$의 차이가 0에 가깝다라는 의미이고 우리가 최소화 해야하는 값입니다.

즉 우리의 목적함수는 프로베니우스 노름(Frobenius norm) $||X-WH||^{2}_{F}$ 정의할 수 있습니다. 

$$min(||X-WH||^{2}_{F})$$

즉 프로베니우스 노름(놈)이 0에 가까워야 하는 것이며 거리(유클리드)가 0에 가깝다는 것입니다.

우리의 목적 함수를 정의하였고 경사하강법을 이용하여 $W$, $H$업데이트 할수 있습니다.

위의 [참고링크1](https://angeloyeo.github.io/2020/10/15/NMF.html)을 보면 $W$, $H$ 행렬을 업데이트하는 과정이 상세하게 잘 나와 있습니다.


결론만 가지고 오면

$$H \Leftarrow H \circ \frac{W^{T}X}{W^{T}WH}$$
$$W \Leftarrow W \circ \frac{XH^{T}}{WHH^{T}}$$  


여기서 $\circ$ 기호는 원소별 곱(Hadamard product)를 의미합니다.


$\epsilon \approx 10^{-9}$와 같은 작은 값으로 분모가 0이 되지 않게 하여 안정성을 높입니다. $U$, $V$가 업데이트 될때 모든 항목이 "동시에" 업데이트가 됩니다.  
그리고 초기화 단계에서는 (0, 1)의 임의의 값으로 초기화 됩니다.

$$H \Leftarrow H \circ \frac{W^{T}X}{W^{T}WH + \epsilon}$$
$$W \Leftarrow W \circ \frac{XH^{T}}{WHH^{T}+ \epsilon}$$  


In [105]:
class NMF_CF:
  def __init__(self, R, hyper_params):
    self.R =np.array(R)
    self.num_users, self.num_items = R.shape
    self.K = hyper_params['K']
    self.iterations =hyper_params['iterations']
    self.alpha = hyper_params['alpha']
    self.beta = hyper_params['beta']
  def fit(self):
    np.random.seed(42)
    self.W = np.random.uniform(0, 1, size=(self.num_users, self.K))
    self.H = np.random.uniform(0, 1, size=(self.K, self.num_items))
    
    
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])

    rows, columns = self.R.shape
    rows = [i for i in range(rows)]
    columns = [i for i in range(columns)]
    self.samples = [(i,j, self.R[i, j]) for i, j in zip(rows, columns)]

    costs = []
    e = 1e-10
    for n in range(self.iterations):
      for i, j, r in self.samples:
        prediction = self.get_prediction(i, j)
        e = (r-prediction)
        
        self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
        self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))
      #Update H
      self.update_H()
      # Update W
      self.update_W()
      cost = self.cost()
      costs.append(cost)
    return costs
  def get_prediction(self, i, j):
    prediction = self.b + self.b_u[i] + self.b_d[j] + self.W[i, :].dot(self.H[:, j])
    return prediction

  def cost(self):
    xs, ys = self.R.nonzero()
    errors = []
    for x, y in zip(xs, ys):
      prediction = self.get_prediction(x, y)
      errors.append(self.R[x, y] - prediction)
    errors = np.array(errors)
    return np.sqrt(np.mean(errors ** 2))

  def update_H(self):
    e = 1e-10
    W_TA = self.W.T @ self.R 
    W_TWH = self.W.T @ self.W @ self.H
    for i in range(self.K):
        for j in range(self.num_items):
              self.H[i, j] = self.H[i, j] * W_TA[i, j] / (W_TWH[i, j] + e)
  def update_W(self):
    e = 1e-10
    AH_T = self.R @ self.H.T
    WHH_T =  self.W @ self.H @ self.H.T
    for i in range(self.num_users):
        for j in range(self.K):
              self.W[i, j] = self.W[i, j] * AH_T[i, j] / (WHH_T[i, j] + e)
            
  def full_matrix(self):
    return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.W.dot(self.H)

In [106]:
import numpy as np

R = np.array([[2, 2, 1, 2, 0, 0],
       [2, 3, 3, 3, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [2, 2, 2, 3, 1, 1],
       [0, 0, 0, 1, 1, 1],
       [0, 0, 0, 2, 1, 2]])
hyper_params = {
    "K" : 2,
    "iterations" : 100,
    'beta' : 0.02,
    'alpha' : 0.001
}

nmf = NMF_CF(R,hyper_params)
costs = nmf.fit()

In [107]:
costs[-1]

1.4377704480513935

In [108]:
nmf.full_matrix()

array([[3.03629128, 3.31535807, 3.11840536, 3.33530232, 1.52410002,
        1.54698739],
       [3.91044028, 4.3809767 , 4.06648878, 4.34310296, 1.46142194,
        1.47304568],
       [2.3371739 , 2.48319304, 2.36930253, 2.44717423, 1.43555145,
        1.43506391],
       [3.23555248, 3.51189438, 3.30249833, 4.5091316 , 2.14712805,
        2.48708608],
       [1.52448581, 1.43576409, 1.45107756, 2.50089   , 2.20445262,
        2.55214198],
       [1.56163905, 1.43560994, 1.46235664, 3.30531025, 2.75759148,
        3.36494142]])